# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import time


In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = student_data.shape[1] - 1
n_passed = len(student_data[student_data.passed == 'yes'])
n_failed = len(student_data[student_data.passed == 'no'])
grad_rate = n_passed / (n_passed + n_failed) * 100
print("Total number of students: %d" % n_students)
print("Number of students who passed: %d" % n_passed)
print("Number of students who failed: %d" % n_failed)
print("Number of features: %d" % n_features)
print("Graduation rate of the class: %.2f" % grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print("Feature column(s):\n")
print(feature_cols)
print("\nTarget column:")
print(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print("\nFeature values:")
print(X_all.head()) # print the first 5 rows

Feature column(s):

['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column:
passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
y_all = y_all.replace(['yes', 'no'], [1, 0]) # we need to convert the target labels to ones and zeros as well
print("Processed feature columns (%d):\n" % len(X_all.columns))
print(list(X_all.columns))

Processed feature columns (48):

['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train_main = X_all.sample(n = num_train)
y_train_main = y_all[X_train_main.index]

X_test_main = X_all[~X_all.index.isin(list(X_train_main.index))]
y_test_main = y_all[~y_all.index.isin(list(y_train_main.index))]
print("Training set: %s samples" % X_test_main.shape[0])
print("Test set: %s samples" % y_test_main.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 95 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
### requirements
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn import grid_search


In [8]:
# helper functions

def train_classifier(clf, X_train, y_train):
    print("\nTraining %s..." % (clf.__class__.__name__))
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print("Done!\nTraining time (secs): %.6f" % (end - start))
    
def predict_labels(clf, features, target):
    print("\nPredicting labels using %s" % (clf.__class__.__name__))
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print("Done!\nPrediction time (secs): %.6f" % (end - start))
    return f1_score(target.values, y_pred, pos_label= 1)

def train_predict(clf, X_train, y_train, X_test, y_test):
    print("------------------------------------------")
    print("Training set size: %d" % (len(X_train)))
    train_classifier(clf, X_train, y_train)
    print("F1 score for training set: %.2f" % (predict_labels(clf, X_train, y_train)))
    print("F1 score for test set: %.2f" % (predict_labels(clf, X_test, y_test)))

def get_constant_test_set(percentage):
    
    num_all = student_data.shape[0]
    num_test = int(percentage * num_all)
    
    X_test = X_all.sample(n = num_test)
    y_test = y_all[X_test.index]
    return X_test, y_test

def get_training_sets(percentage, X_test):
    
    potential_training_set = X_all[~X_all.index.isin(list(X_test.index))]
        
    num_all = potential_training_set.shape[0]
    num_train = int(percentage * num_all)
    
    X_train = potential_training_set.sample(n = num_train)
    y_train = y_all[X_train.index]
    return X_train, y_train

def main_function(clf):
    
    train_classifier(clf, X_train_main, y_train_main)  # note: using entire training set here
    print(clf)  # you can inspect the learned model by printing it
    
    train_f1_score = predict_labels(clf, X_train_main, y_train_main)
    print("F1 score for training set: %.2f" % train_f1_score)

    # Predict on test data
    print("F1 score for test set: %.2f" % predict_labels(clf, X_test_main, y_test_main))
    
    for percentage in training_set_percentage_list:
        X_train, y_train = get_training_sets(percentage, X_test)
        train_predict(clf, X_train, y_train, X_test, y_test)

In [9]:
### define models to be trained
clf_decisiontree = tree.DecisionTreeClassifier()
clf_randomforest = RandomForestClassifier(n_estimators=10)
clf_svm = svm.SVC()

In [10]:
# train and predict with models

model_list = [clf_decisiontree, clf_randomforest, clf_svm]

X_test, y_test = get_constant_test_set(0.1)

training_set_percentage_list = [0.3, 0.5, 0.7, 1] # We can use 1, i.e. 100%, of the training set, because we are already excluding 
                                                  # the test data before we selecting the test cases

for model in model_list:
    main_function(model)



Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003231
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')

Predicting labels using DecisionTreeClassifier
Done!
Prediction time (secs): 0.000429
F1 score for training set: 1.00

Predicting labels using DecisionTreeClassifier
Done!
Prediction time (secs): 0.000226
F1 score for test set: 0.76
------------------------------------------
Training set size: 106

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.000769

Predicting labels using DecisionTreeClassifier
Done!
Prediction time (secs): 0.000158
F1 score for training set: 1.00

Predicting labels using DecisionTreeClassifier
Done!
Prediction time (secs): 0.000216
F1 score for test set: 0.79
------------------------------------------
Training set size:

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [11]:
# TODO: Fine-tune your model and report the best F1 score

# We will finetune our SVM. We will test the following params: cost (C), gamma, and kernel. 

percentage = 1

param_grid = [
  {'C': [10, 100, 1000], 'kernel': ['linear']},
  {'C': [10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

X_test, y_test = get_constant_test_set(0.1)
X_train, y_train = get_training_sets(percentage, X_test)

clf_svm = svm.SVC()
clf = grid_search.GridSearchCV(clf_svm, param_grid, n_jobs = 4, scoring = "f1")
clf.fit(X_train, y_train)


print("F1 score for test set: %.2f" % (predict_labels(clf, X_test, y_test)))


Predicting labels using GridSearchCV
Done!
Prediction time (secs): 0.000762
F1 score for test set: 0.75


In [12]:
print("Best parameters set found on development set:\n")
print(clf.best_params_)
print("\nGrid scores on development set:\n")
for params, mean_score, scores in clf.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() * 2, params))
print()

Best parameters set found on development set:

{'gamma': 0.0001, 'kernel': 'rbf', 'C': 100}

Grid scores on development set:

0.784 (+/-0.102) for {'kernel': 'linear', 'C': 10}
0.780 (+/-0.098) for {'kernel': 'linear', 'C': 100}
0.780 (+/-0.098) for {'kernel': 'linear', 'C': 1000}
0.817 (+/-0.007) for {'gamma': 0.001, 'kernel': 'rbf', 'C': 10}
0.807 (+/-0.008) for {'gamma': 0.0001, 'kernel': 'rbf', 'C': 10}
0.798 (+/-0.029) for {'gamma': 0.001, 'kernel': 'rbf', 'C': 100}
0.820 (+/-0.028) for {'gamma': 0.0001, 'kernel': 'rbf', 'C': 100}
0.779 (+/-0.065) for {'gamma': 0.001, 'kernel': 'rbf', 'C': 1000}
0.802 (+/-0.074) for {'gamma': 0.0001, 'kernel': 'rbf', 'C': 1000}



In [13]:
from sklearn.metrics import classification_report

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))


             precision    recall  f1-score   support

          0       1.00      0.06      0.12        16
          1       0.61      1.00      0.75        23

avg / total       0.77      0.62      0.49        39



In [14]:
print(clf.get_params())

{'iid': True, 'refit': True, 'fit_params': {}, 'estimator__C': 1.0, 'estimator__random_state': None, 'n_jobs': 4, 'estimator__tol': 0.001, 'estimator__kernel': 'rbf', 'estimator__verbose': False, 'verbose': 0, 'estimator__gamma': 0.0, 'param_grid': [{'kernel': ['linear'], 'C': [10, 100, 1000]}, {'gamma': [0.001, 0.0001], 'kernel': ['rbf'], 'C': [10, 100, 1000]}], 'loss_func': None, 'estimator__shrinking': True, 'estimator__cache_size': 200, 'scoring': 'f1', 'estimator': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False), 'estimator__coef0': 0.0, 'estimator__class_weight': None, 'estimator__degree': 3, 'score_func': None, 'cv': None, 'pre_dispatch': '2*n_jobs', 'estimator__max_iter': -1, 'estimator__probability': False, 'error_score': 'raise'}
